In [ ]:
import logging
import pickle
import dill
import numpy as np
import matplotlib.pyplot as plt

from src.emulator import Emulator
from src.emulator_BAND import EmulatorBAND

In [ ]:
def loadEmulator(emulatorPathList):
        emuList = []
        for i, emuPath in enumerate(emulatorPathList):
            with open(emuPath, 'rb') as f:
                emu_i = dill.load(f)
                emuList.append(emu_i)
        logging.info("Number of Emulators: {}".format(len(emuList)))
        return emuList

In [ ]:
folder = "test_mcmc_unpacked_emu/"
emuPathList = [folder+"AuAu200_dNdy_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu200_pTvn_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu200_PHOBOSdNdeta_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu200_PHOBOSv2eta_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu19p6_dNdy_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu19p6_pTvn_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu19p6_PHOBOSdNdeta_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu7.7_dNdy_train_PCA_noLOG_PCSK_trained.sav",
               folder+"AuAu7.7_pTvn_train_PCA_noLOG_PCSK_trained.sav",
            ]

emu_list = loadEmulator(emuPathList)

In [ ]:
file_path = './test_data/example_data_test_point1099.pkl'

with open(file_path,'rb') as file:
    loaded_data = pickle.load(file)

In [ ]:
for key in loaded_data.keys():
    a = key

num_obs = len(loaded_data[a]['obs'][0])
num_param = len(loaded_data[a]['parameter'])

response_matrix = np.zeros((num_obs, num_param))

X = np.array(loaded_data[a]['parameter'], copy = False, ndmin = 2)

for j in range(num_param):
    print("j is ", j)
    X_h_1 = np.copy(X)
    X_h_2 = np.copy(X)
    h = 0.1*X_h_1[0,j]
    X_h_1[0, j] = X_h_1[0, j] + h
    X_h_2[0, j] = X_h_2[0, j] - h

    Y_h_1 = []
    Y_h_2 = []
    for emu in emu_list:
        Y_h_1.append(emu.predict(X_h_1, return_cov = False))
        Y_h_2.append(emu.predict(X_h_2, return_cov = False))
    Y_h_1_ = np.concatenate(Y_h_1, axis = 1)
    Y_h_2_ = np.concatenate(Y_h_2, axis = 1)
    
    response_matrix[:, j] = (Y_h_1_[0,:] - Y_h_2_[0,:])/(2*h)*X[0, j]/(Y_h_1_[0,:]+Y_h_2_[0,:])*2
        

In [ ]:
PARAMETER_NAMES = [
r'$B_G\;[\mathrm{GeV}^{-2}]$',
r'$\alpha_{\rm shadowing}$',
r'$y_{{\rm loss},2}$',
r'$y_{{\rm loss},4}$',
r'$y_{{\rm loss},6}$',
r'$\sigma_{y_{\rm loss}}$',
r'$\alpha_{\rm rem}$',
r'$\lambda_B$',
r'$\sigma_x^{\rm string}\;[{\rm fm}]$',
r'$\sigma_\eta^{\rm string}$',
r'$\alpha_{{\rm string}\;{\rm tilt}}$',
r'$\alpha_{\rm preFlow}$',
r'$\eta_0$',
r'$\eta_2$',
r'$\eta_4$',
r'$\zeta_{\rm max}$',
r'$T_{\zeta,0}\;[{\rm GeV}]$',
r'$\sigma_{\zeta,+}\;[{\rm GeV}]$',
r'$\sigma_{\zeta,-}\;[{\rm GeV}]$',
r'$e_{\rm sw}\;[{\rm GeV}/{\rm fm}^3]$'
]

x = np.arange(len(PARAMETER_NAMES))

In [ ]:
min_value = np.amin(response_matrix)
max_value = np.amax(response_matrix)
c_value = max(min_value, max_value)

height, width = response_matrix.shape
aspect_ratio = 3.4*((width)/(height))

# Split the response matrix into two parts
half_width = width // 2
response_matrix_1, response_matrix_2 = np.hsplit(response_matrix, [half_width])

plt.figure(figsize=(10, 20))
plt.imshow(response_matrix, cmap='coolwarm', vmin=-0.1, vmax=0.1, aspect = aspect_ratio, interpolation = 'nearest')
# Add color bar
plt.colorbar(shrink=0.5)  # Adjust shrink value as needed

xtickPos = [3, 10, 17, 24, 31, 38, 45, 52, 58, 64, 83, 116, 152, 189, 223,
            245, 254, 262, 268, 276, 283, 290, 297, 303, 310, 330, 364,
            398, 433, 468, 486, 493, 500, 507, 514, 521, 528, 534, 540]
xticksLabel = [r"$dN/dy(\pi^+)$",
               r"$dN/dy(K^+)$", r"$dN/dy(p)$",
               r"$dN/dy(\bar{p})$", r"$\langle p_T \rangle(\pi^+)$",
               r"$\langle p_T \rangle(K^+)$",
               r"$\langle p_T \rangle(p)$",
               r"$\langle p_T \rangle(\bar{p})$",
               r"$v_2\{2\}({\rm ch})$",
               r"$v_3\{2\}({\rm ch})$",
               r"$dN^{\rm ch}/d\eta(0-5\%)$",
               r"$dN^{\rm ch}/d\eta(5-12\%)$",
               r"$dN^{\rm ch}/d\eta(12-24\%)$",
               r"$dN^{\rm ch}/d\eta(24-34\%)$",
               r"$dN^{\rm ch}/d\eta(34-44\%)$",
               r"$v_2(\eta)(0-40\%)$",
               r"$dN/dy(\pi^+)$",
               r"$dN/dy(K^+)$", r"$dN/dy(p)$",
               r"$\langle p_T \rangle(\pi^+)$",
               r"$\langle p_T \rangle(K^+)$",
               r"$\langle p_T \rangle(p)$",
               r"$\langle p_T \rangle(\bar{p})$",
               r"$v_2\{2\}({\rm ch})$",
               r"$v_3\{2\}({\rm ch})$",
               r"$dN^{\rm ch}/d\eta(0-5\%)$",
               r"$dN^{\rm ch}/d\eta(5-12\%)$",
               r"$dN^{\rm ch}/d\eta(12-24\%)$",
               r"$dN^{\rm ch}/d\eta(24-34\%)$",
               r"$dN^{\rm ch}/d\eta(34-44\%)$",
               r"$dN/dy(\pi^+)$",
               r"$dN/dy(K^+)$", r"$dN/dy(p)$",
               r"$\langle p_T \rangle(\pi^+)$",
               r"$\langle p_T \rangle(K^+)$",
               r"$\langle p_T \rangle(p)$",
               r"$\langle p_T \rangle(\bar{p})$",
               r"$v_2\{2\}({\rm ch})$",
               r"$v_3\{2\}({\rm ch})$",
             ]

ax = plt.gca()
ax.xaxis.tick_top()
ax.set_yticks(xtickPos)
ax.set_yticklabels(xticksLabel, rotation=0)
plt.yticks(fontsize = 14)
plt.xticks(x, PARAMETER_NAMES, fontsize=14, rotation = 90)
plt.tight_layout()
plt.savefig("heatmap_1099.pdf")
plt.show()


In [ ]:
# Assuming you have already defined PARAMETER_NAMES and response_matrix

min_value = np.amin(response_matrix)
max_value = np.amax(response_matrix)
c_value = max(min_value, max_value)

height, width = response_matrix.shape
aspect_ratio = 2 * (width / height)

# Split the matrix by its 252nd element
split_index = 251
response_matrix_1 = response_matrix[:split_index]
response_matrix_2 = response_matrix[split_index:]

# Plotting
fig, axs = plt.subplots(1, 2, figsize=(28, 40))

# Plot the first part
im1 = axs[0].imshow(response_matrix_1, cmap='coolwarm', vmin=-0.1, vmax=0.1, aspect='auto', interpolation='nearest')

# Plot the second part
im2 = axs[1].imshow(response_matrix_2, cmap='coolwarm', vmin=-0.1, vmax=0.1, aspect='auto', interpolation='nearest')

# Create a color bar for both subplots
cbar = fig.colorbar(im1, ax=axs, orientation='horizontal')
cbar.ax.set_position([0.15, -0.07, 0.7, 1])
cbar.ax.tick_params(labelsize=35)

# Set ticks and labels for x-axis
axs[0].set_xticks(x)
axs[1].set_xticks(x)
axs[0].set_xticklabels(PARAMETER_NAMES, fontsize=35, rotation=90)
axs[1].set_xticklabels(PARAMETER_NAMES, fontsize=35, rotation=90)

# Set ticks and labels for y-axis with adjusted spacing
yticks_1 = xtickPos[:16]
yticks_2 = [x - 252 for x in xtickPos[-23:]]
axs[0].set_yticks(yticks_1)
axs[1].set_yticks(yticks_2)
axs[0].set_yticklabels(xticksLabel[:16], fontsize=35)
axs[1].set_yticklabels(xticksLabel[-23:], fontsize=35)
# Set yticks on the right side for the second graph
axs[1].tick_params(axis='y', labelright=True, labelleft=False)

# Save or display the plot
plt.savefig("heatmap_1099_split.pdf", bbox_inches='tight')
plt.show()
